<a href="https://colab.research.google.com/github/ulilazmi100/LLMs/blob/main/Quantize_Llama_2_models_using_GGUF_and_llama_cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantize Llama 2 models using GGUF and llama.cpp
> 🗣️ [Large Language Model Course](https://github.com/mlabonne/llm-course)

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne).

## Usage

* `MODEL_ID`: The ID of the model to quantize (e.g., `mlabonne/EvolCodeLlama-7b`).
* `QUANTIZATION_METHOD`: The quantization method to use.

## Quantization methods

The names of the quantization methods follow the naming convention: "q" + the number of bits + the variant used (detailed below). Here is a list of all the possible quant methods and their corresponding use cases, based on model cards made by [TheBloke](https://huggingface.co/TheBloke/):

* `q2_k`: Uses Q4_K for the attention.vw and feed_forward.w2 tensors, Q2_K for the other tensors.
* `q3_k_l`: Uses Q5_K for the attention.wv, attention.wo, and feed_forward.w2 tensors, else Q3_K
* `q3_k_m`: Uses Q4_K for the attention.wv, attention.wo, and feed_forward.w2 tensors, else Q3_K
* `q3_k_s`: Uses Q3_K for all tensors
* `q4_0`: Original quant method, 4-bit.
* `q4_1`: Higher accuracy than q4_0 but not as high as q5_0. However has quicker inference than q5 models.
* `q4_k_m`: Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K
* `q4_k_s`: Uses Q4_K for all tensors
* `q5_0`: Higher accuracy, higher resource usage and slower inference.
* `q5_1`: Even higher accuracy, resource usage and slower inference.
* `q5_k_m`: Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K
* `q5_k_s`:  Uses Q5_K for all tensors
* `q6_k`: Uses Q8_K for all tensors
* `q8_0`: Almost indistinguishable from float16. High resource use and slow. Not recommended for most users.

As a rule of thumb, **I recommend using Q5_K_M** as it preserves most of the model's performance. Alternatively, you can use Q4_K_M if you want to save some memory. In general, K_M versions are better than K_S versions. I cannot recommend Q2_K or Q3_* versions, as they drastically decrease model performance.

In [ ]:
# Variables
MODEL_ID = "mlabonne/EvolCodeLlama-7b"
QUANTIZATION_METHODS = ["q4_k_m"]

# Constants
MODEL_NAME = MODEL_ID.split('/')[-1]

In [1]:
# Install llama.cpp
!git clone https://github.com/ggerganov/llama.cpp || echo "llama.cpp directory already exists"
!cd llama.cpp && git pull

# Clean up any previous build artifacts
!cd llama.cpp && make clean

# Build the specific binaries for 'main' and 'llama-quantize' with CUDA support if needed
!cd llama.cpp && make GGML_CUDA=1 main
!cd llama.cpp && make GGML_CUDA=1 llama-quantize

# Install Python dependencies
!pip install -r llama.cpp/requirements.txt

Cloning into 'llama.cpp'...
remote: Enumerating objects: 33232, done.
remote: Counting objects: 100% (8651/8651), done.
remote: Compressing objects: 100% (767/767), done.
remote: Total 33232 (delta 8252), reused 8004 (delta 7853), pack-reused 24581 (from 1)
Receiving objects: 100% (33232/33232), 57.14 MiB | 10.00 MiB/s, done.
Resolving deltas: 100% (24023/24023), done.
Already up to date.
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE  -std=c11   -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -g 

In [ ]:
# Download model
!git lfs install
!git clone https://huggingface.co/{MODEL_ID}

Git LFS initialized.
Cloning into 'EvolCodeLlama-7b'...
remote: Enumerating objects: 35, done.
remote: Total 35 (delta 0), reused 0 (delta 0), pack-reused 35 (from 1)
Unpacking objects: 100% (35/35), 483.38 KiB | 5.89 MiB/s, done.


In [5]:
# Convert to fp16
fp16 = f"{MODEL_NAME}/{MODEL_NAME.lower()}.fp16.bin"
!python ./llama.cpp/convert_hf_to_gguf.py {MODEL_NAME} --outtype f16 --outfile {fp16}

INFO:hf-to-gguf:Loading model: EvolCodeLlama-7b
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00002.bin'
Traceback (most recent call last):
  File "/content/./llama.cpp/convert_hf_to_gguf.py", line 4149, in <module>
    main()
  File "/content/./llama.cpp/convert_hf_to_gguf.py", line 4143, in main
    model_instance.write()
  File "/content/./llama.cpp/convert_hf_to_gguf.py", line 394, in write
    self.prepare_tensors()
  File "/content/./llama.cpp/convert_hf_to_gguf.py", line 1605, in prepare_tensors
    super().prepare_tensors()
  File "/content/./llama.cpp/convert_hf_to_gguf.py", line 265, in prepare_tensors
    for name, data_torch in self.get_tensors():
  File "/content/./llama.cpp/convert_hf_to_gguf.py", line 156, in get_tensors
    ctx = contextlib.nullcontext(torch.load(str(

In [3]:
# Quantize the model for each method in the QUANTIZATION_METHODS list
for method in QUANTIZATION_METHODS:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{method.upper()}.gguf"
    !./llama.cpp/llama-quantize {fp16} {qtype} {method}

./llama.cpp/llama-quantize: error while loading shared libraries: libcuda.so.1: cannot open shared object file: No such file or directory


## Run inference

Here is a simple script to run your quantized models. I'm offloading every layer to the GPU (35 for a 7b parameter model) to speed up inference.

In [4]:
import os

model_list = [file for file in os.listdir(MODEL_NAME) if "gguf" in file]

print(model_list)

prompt = input("Enter your prompt: ")
chosen_method = input("Name of the model (options: " + ", ".join(model_list) + "): ")

# Verify the chosen method is in the list
if chosen_method not in model_list:
    print("Invalid name")
else:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{method.upper()}.gguf"
    !./llama.cpp/main -m {qtype} -n 128 --color -ngl 35 -p "{prompt}"

[]


KeyboardInterrupt: Interrupted by user

## Push to hub

To push your model to the hub, you'll need to input your Hugging Face token (https://huggingface.co/settings/tokens) in Google Colab's "Secrets" tab. The following code creates a new repo with the "-GGUF" suffix. Don't forget to change the `username` variable.

In [ ]:
!pip install -q huggingface_hub
from huggingface_hub import create_repo, HfApi
from google.colab import userdata

username = "mlabonne"

# Defined in the secrets tab in Google Colab
api = HfApi(token=userdata.get("HF_TOKEN"))

# Create empty repo
create_repo(
    repo_id = f"{username}/{MODEL_NAME}-GGUF",
    repo_type="model",
    exist_ok=True,
)

# Upload gguf files
api.upload_folder(
    folder_path=MODEL_NAME,
    repo_id=f"{username}/{MODEL_NAME}-GGUF",
    allow_patterns=f"*.gguf",
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.8 MB/s eta 0:00:00
